In [5]:
import pandas as pd
import numpy as np
import time
from sklearn import linear_model
from datetime import datetime
import pytz

In [6]:
cd C:\Users\nikhi\Desktop\New folder

C:\Users\nikhi\Desktop\New folder


In [7]:
import logging
from kiteconnect import KiteConnect
logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="offq4l2e3rytdipj")

In [8]:
kite.login_url()

'https://kite.zerodha.com/connect/login?api_key=offq4l2e3rytdipj&v=3'

In [9]:
data = kite.generate_session("RFCXITeLg5DfhcKdJyOxccagB3fDFbRq", api_secret="3xaieymiwahzi2qsvtyecni9ddl42jui")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


In [10]:
kite.set_access_token(data["access_token"])
print ("---------------------------")
print ("Access Token:", data["access_token"])

---------------------------
Access Token: 4r8UoCpSKUuLm42p4vjIbTMdkt4GMZI2


In [11]:
varty=kite.VARIETY_REGULAR
scrip='IDBI'
prd_type=kite.PRODUCT_MIS
qty=1
exchg=kite.EXCHANGE_NSE
ord_typ=kite.ORDER_TYPE_MARKET
valdty=kite.VALIDITY_DAY
exchange='NSE'


In [12]:
def buyorder():
    try:
        order_id = kite.place_order(variety=varty,
                                    tradingsymbol=scrip,
                                    exchange=exchg,
                                    transaction_type=kite.TRANSACTION_TYPE_BUY,
                                    quantity=qty,
                                    order_type=ord_typ,
                                    product=prd_type,
                                    validity=valdty)
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
    
def sellorder():
    try:
        order_id = kite.place_order(variety=varty,
                                    tradingsymbol=scrip,
                                    exchange=exchg,
                                    transaction_type=kite.TRANSACTION_TYPE_SELL,
                                    quantity=qty,
                                    order_type=ord_typ,
                                    product=prd_type,
                                    validity=valdty)
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))        

In [13]:
def LTP_f():
    LTP=kite.ltp('NSE:IDBI')
    a=LTP
    return (a['NSE:IDBI']['last_price'])

In [14]:
#for 1 position scrip

def buyprice ():
    d=kite.positions()
    [d1]=(d['net'])
    BUYPRICE=d1['buy_price']
    return BUYPRICE

def sellprice ():
    s=kite.positions()
    [s1]=(s['net'])
    SELLPRICE=s1['sell_price']
    return SELLPRICE

def pnl ():
    d=kite.positions()
    [pn]=(d['net'])
    PNL=pn['pnl']
    return PNL

def pqty():
    d=kite.positions()
    [pq]=(d['net'])
    P_qty=pq['quantity']
    return  P_qty

def ptime():
    # Get the timezone object for London
    tz_India = pytz.timezone('Asia/Calcutta')

    # Get the current time in London
    datetime_India = datetime.now(tz_India)

    # Format the time as a string and print it
    print("India time:", datetime_India.strftime("%H:%M:%S"))
    
def Order_Status():
    v=kite.orders()
    OID=v[:][-1]['order_id']
    status=v[:][-1]['status']
    return status

In [15]:
def Last_close():
    instrument_token=377857
    from_date='2023-02-07'
    to_date='2023-02-21'
    interval='minute'
    his=kite.historical_data(instrument_token, from_date, to_date, interval, continuous=False, oi=False)
    df = pd.DataFrame(data=his)
    L_close=df['close'].iloc[-1]
    return L_close

In [16]:
def MA():
    instrument_token=377857
    from_date='2023-02-07'
    to_date='2023-02-21'
    interval='minute'
    his=kite.historical_data(instrument_token, from_date, to_date, interval, continuous=False, oi=False)
    df = pd.DataFrame(data=his)
    y=df['close'].to_frame()
    MA= y['close'].ewm(span=5).mean()
    MA1=y.iloc[-1].values
    return MA1

In [17]:
def LTP_MA():
    LTP_MA=[]
    for i in range(5):
        LTP_MA.append(LTP_f())
        df2 = pd.DataFrame (LTP_MA)
        time.sleep(3)
        MA=df2.ewm(span=5).mean()
        MA1=MA.iloc[-1].values
    return MA1.round(2)

In [18]:
def z_pos():
    d=kite.positions()
    ops=(d['net'])
    if len(ops)==0:
        return (0)
    else:
        return (1)


In [19]:
def bid():
    b=kite.quote('NSE:IDBI')
    bq=b['NSE:IDBI']['buy_quantity']
    sq=b['NSE:IDBI']['sell_quantity']
    if bq>sq:
        bid='positive'
    else:
        bid='negetive'
    return bid

In [20]:
def predicted_price():
    instrument_token=377857
    from_date='2023-02-07'
    to_date='2023-02-21'
    interval='minute'
    his=kite.historical_data(instrument_token, from_date, to_date, interval, continuous=False, oi=False)
    df = pd.DataFrame(data=his)
    x=df[['open','high','low']]
    y=df['close']
    reg=linear_model.LinearRegression()
    reg.fit(x,y)
    Last_row=x.iloc[-1:,[0,1,2]].values
    Last_row
    pred=reg.predict(Last_row)
    return pred

In [22]:
#begin trade
for i in range(30):
    if LTP_MA()>Last_close() and z_pos()==0:
        buyorder()
        print("Buy order")
        break
       
    elif LTP_MA()<Last_close() and z_pos()==0:
        #sellorder()
        print("sell order-shortsell")
        break
        
    else:
            print("Trade not started")
    ptime()
    time.sleep(30)

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/377857/minute?from=2023-02-07&to=2023-02-21&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 47424
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP

sell order-shortsell


In [29]:
#highselling
while bid()=='positive':
    count=(0)
    for i in range(60):

        if pqty()==0 and pnl ()>-15 and LTP_MA()>Last_close() and Order_Status()=='COMPLETE':
                buyorder()
                print("Buy order")
        elif pqty()==qty and LTP_MA()>0.10+buyprice():
                sellorder()
                print("sellorder-profit achieved")
        elif pqty()==qty and LTP_MA()>predicted_price() and LTP_MA()<0.10+buyprice():
                print("Hold")      
        elif pqty()==qty and LTP_MA()<0.10-buyprice():
                sellorder()
                print("sellorder")
        elif pqty()==qty and pnl ()<=-15:
                sellorder()
                print("sellorder-PNL<=0")
        elif pqty()==0 and pnl ()<=-15:
                print("Close for the day-PNL<=0")

        else:
                print("No trades placed..")

        
        count+=1
        print('count:', count)
        ptime()
        time.sleep(60)


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NSE%3AIDBI HTTP/1.1" 200 None


In [32]:
buyorder()

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /orders/regular HTTP/1.1" 200 None
INFO:root:Order placed. ID is: 230221201277799


In [46]:
while bid()=='positive':
    count=(0)
    for i in range(60):
        ltp_ma=LTP_MA()
        if pqty()==0 and pnl ()>-15 and ltp_ma>Last_close() and Order_Status()=='COMPLETE':
                buyorder()
                print("Buy order")
        elif pqty()==qty and ltp_ma>0.10+buyprice():
                sellorder()
                print("sellorder-profit achieved")
        elif pqty()==qty and ltp_ma>predicted_price() and ltp_ma<0.10+buyprice():
                print("Hold")      
        elif pqty()==qty and ltp_ma<buyprice()-0.10:
                sellorder()
                print("sellorder")
        elif pqty()==qty and pnl ()<=-15:
                sellorder()
                print("sellorder-PNL<=0")
        elif pqty()==0 and pnl ()<=-15:
                print("Close for the day-PNL<=0")

        else:
                print("No trades placed..")

        
        count+=1
        print('count:', count)
        ptime()
    time.sleep(60)


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/hi

Hold
count: 1
India time: 13:55:02


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/hi

Hold
count: 2
India time: 13:56:20


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/hi

...

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/377857/minute?from=2023-02-07&to=2023-02-21&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 46913
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200

No trades placed..
count: 43
India time: 14:49:05


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3AIDBI HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/377857/minute?from=2023-02-07&to=2023-02-21&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 46926
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200

No trades placed..
count: 44
India time: 14:50:22


KeyboardInterrupt: 